In [1]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

#Imports necessários
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = pd.read_csv('C:\\Users\\ANA GABRIELA\\Documents\\Python Scripts\\PUC\\Machine Learning\\CreditRisk.data', sep =',', header = None)

In [3]:
dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


In [4]:
# Entendimento estatístico do dataset
dataset_description = dataset.describe()
print(dataset_description)

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


In [5]:
#Metadados

#A1:	b, a.
#A2:	continuous.
#A3:	continuous.
#A4:	u, y, l, t.
#A5:	g, p, gg.
#A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
#A7:	v, h, bb, j, n, z, dd, ff, o.
#A8:	continuous.
#A9:	t, f.
#A10:	t, f.
#A11:	continuous.
#A12:	t, f.
#A13:	g, p, s.
#A14:	continuous.
#A15:	continuous.
#A16: +,-         (class attribute)

# Entendimento do data type das colunas
dataset_info = dataset.info()
print(dataset_info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB
None


In [6]:
#É possível notar que o dtype das colunas 1 e 13 não conversa com os metadados da base de dados. No entanto, ao tentar alterar as colunas não foi possível, pois apresenta caracter string. Iremos investigar; 
print(dataset.columns)

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')


In [7]:
# Primeiros 20 valores do dataset. Não foi possível identificar o erro.
dataset.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.040,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.500,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.750,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.710,t,f,0,f,s,00120,0,+
5,b,32.08,4.000,u,g,m,v,2.500,t,f,0,t,g,00360,0,+
6,b,33.17,1.040,u,g,r,h,6.500,t,f,0,t,g,00164,31285,+
7,a,22.92,11.585,u,g,cc,v,0.040,t,f,0,f,g,00080,1349,+
8,b,54.42,0.500,y,p,k,h,3.960,t,f,0,f,g,00180,314,+
9,b,42.50,4.915,y,p,w,v,3.165,t,f,0,t,g,00052,1442,+


In [8]:
# Últimos 20 valores do dataset. Foi possível identificar o caracter "?" na coluna 1, o que impede a conversão para numérico.
dataset.tail(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


In [9]:
# Substituição de '?' por NaN
dataset = dataset.replace('?', np.nan)

# Check se houve a substituição
dataset.tail(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


In [10]:
# Subsitutição de Nan pela média para numéricos
dataset.fillna(dataset.mean(), inplace=True)

# Check se ainda possui Nan
dataset.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

In [11]:
#As colunas numéricas foram preenchinas, mas as categóricas ainda possuem nulos

# Iterate todas as colunas
for col in dataset.columns:
    # Apenas as colunas categóricas
    if dataset[col].dtypes == 'object':
        # Preencher com o valor que mais aparece
        dataset = dataset.fillna(dataset[col].value_counts().index[0])

# Check se todas as colunas foram preenchidas
print(dataset.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


In [12]:
# LabelEncoder para converter dados categóricos em numéricos
le = LabelEncoder() 

# Iterate para passar por todas as colunas e identificar as categóricas
for col in dataset.columns.to_numpy():
    if dataset[col].dtypes =='object':
    # LabelEncoder para transformar em dado numérico
        dataset[col]=le.fit_transform(dataset[col])

In [13]:
#Agora é possível converter as colunas 1 e 13 em variável contínua
dataset[[1, 13]] = dataset[[1, 13]].astype(float)

In [14]:
# Retirar features 11 and 13 (habilitação e ZIPCODE) e converter o DataFrame em Numpy
dataset = dataset.drop([11, 13], axis=1)
dataset = dataset.to_numpy()

# SSeparar features e linhas em variáveis separadas
X,y = dataset[:,0:13] , dataset[:,13]

# Dividir em conjunto treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,
                                y,
                                test_size=0.33,
                                random_state=42)

In [15]:
# Instanciar o MinMaxScaler e usá-lo para reescalar X_train e X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [16]:
# Instanciar um classificador de regressão logística com valores padrão de parâmetros
logreg = LogisticRegression()

#Aplicando a regressão logistica ao cojunto de treino
logreg.fit(rescaledX_train, y_train)

LogisticRegression()

In [18]:
# Aplicando ao conjunto de teste
y_pred = logreg.predict(rescaledX_test)

# Acuracidade do modelo
print("Acuracidade da Regressão Logística: ", logreg.score(rescaledX_test, y_test))

# Matriz de confusão da regressão logística
print(confusion_matrix(y_test, y_pred))

Acuracidade da Regressão Logística:  0.8421052631578947
[[94  9]
 [27 98]]


In [19]:
#Para aumentar a acuracidade do modelo, aplicaremos a regularização de Lasso:
# Configuração de regularização Lasso
logreg_lasso = LogisticRegression(penalty='l1', solver='liblinear')

# Usar o scaler para reescalar as variáveis
rescaledX = scaler.fit_transform(X)

# Busca por hiperparâmetros 
param_grid = {'C': [0.01, 0.1, 1, 10]}
grid_model = GridSearchCV(estimator=logreg_lasso, param_grid=param_grid, cv=5)

# Ajustar os dados ao modelo
grid_model_result = grid_model.fit(rescaledX, y)

# Resultados
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.855072 using {'C': 0.1}
